Neste exemplo você ira construir um classificador binário para o dataset IMDB (NLP Problem)

O imdb dataset já vem no Keras e já é pre-processado. Os Reviews (sequencia de palavras) já estão organizados em sequências de inteiros, em que cada inteiro significa uma palavra específica do dicionário.

In [1]:
from keras.datasets import imdb

As 10000 palavras mais frequentes serão mantidas no dataset

In [2]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 1s 0us/step
<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
D:\Users\angel\Anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
D:\Users\angel\Anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or nda

train_data e test_data são as listas de reviews, cada review é uma lista de indices de palavras (texto do review codificado no indice de palavra)

In [3]:
print(train_data[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


train_labels e test_labels são as listas de 0 e 1 (0=review negativo, 1=review positivo)

In [4]:
print(train_labels[0])

1


Veja que nenhum indice da palavra ira exceder 10000

In [5]:
max([max(sequence) for sequence in train_data])

9999

Não podemos inserir uma lista de numeros inteiros na rede. Temos que converter a lista para um tensor no formato (samples, word_indices).

Por exemplo, transformar a sequencia \[3,5\] em um vetor de 10000 dimensões que seria todos seriam 0s, exceto os indices 3 e 5 seriam 1s.

In [6]:
import numpy as np

In [7]:
def vectorize_sequence(sequence, dimension=10000):
    
    # create an all-zeros matrix of shape (len(sequence), dimension)
    results = np.zeros((len(sequence), dimension))

    for i, sequence in enumerate(sequence):
        results[i, sequence] = 1.

    return results

In [8]:
x_train = vectorize_sequence(train_data)
x_test = vectorize_sequence(test_data)

In [9]:
y_train = np.asarray(train_labels).astype('float')
y_test =  np.asarray(test_labels).astype('float')

In [10]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

**Construindo a rede**

O input é um vetor e os labels são escalares (1s e 0s). Vamos usar um fully connected (Dense) com relu como activation function. Dense(16, activation='relu')

16: é o argumento para cada dense layer. Ou seja, é o número de hidden units da camada (é a dimensão da representação da camada).

**Configuração da rede**

Iremos configurar a rede com duas camadas intermediarias com 16 unidades ocultas cada, que reproduzir-a a previsão escalar em relação ao sentimento do review em questão.

In [12]:
from keras import layers, models

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Escolhendo a loss function e o optimizer (como strings por já fazerem parte do keras)

In [13]:
from keras import losses, metrics
from keras.optimizers import SGD, RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001), loss=losses.binary_crossentropy, metrics=[metrics.binary_accuracy])

No entanto, vode pode deixar tudo como padrão

In [14]:
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['binary_accuracy'])

Para monitorar o treinamento, criamos o conjunto de validação separando 10000 amostras do conjunto original

In [16]:
x_val = x_train[10000:]
partial_x_train = x_train[10000:]
y_val = y_train[10000:]
partial_y_train = y_train[10000:]

Iremos treinar o modelo por 20 epochs (20 interações sobre todas as amostras do treinamento) em um batch de 512 amostras

In [17]:
model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20
30/30 [==============================] - 1s 30ms/step - loss: 0.5412 - binary_accuracy: 0.7605 - val_loss: 0.3778 - val_binary_accuracy: 0.8873
Epoch 2/20
30/30 [==============================] - 0s 15ms/step - loss: 0.3292 - binary_accuracy: 0.8993 - val_loss: 0.2733 - val_binary_accuracy: 0.9126
Epoch 3/20
30/30 [==============================] - 0s 14ms/step - loss: 0.2385 - binary_accuracy: 0.9253 - val_loss: 0.2222 - val_binary_accuracy: 0.9205
Epoch 4/20
30/30 [==============================] - 0s 15ms/step - loss: 0.1862 - binary_accuracy: 0.9420 - val_loss: 0.1506 - val_binary_accuracy: 0.9581
Epoch 5/20
30/30 [==============================] - 0s 14ms/step - loss: 0.1527 - binary_accuracy: 0.9512 - val_loss: 0.1167 - val_binary_accuracy: 0.9703
Epoch 6/20
30/30 [==============================] - 0s 14ms/step - loss: 0.1249 - binary_accuracy: 0.9609 - val_loss: 0.1293 - val_binary_accuracy: 0.9554
Epoch 7/20
30/30 [==============================] - 0s 14ms/step - los

**Usando uma rede treinada para gerar previsões sobre novos dados**

Gerando a probabilidade de cada análise ser positiva com o método predict

In [19]:
print(model.predict(x_test))

[[3.9613247e-03]
 [9.9999875e-01]
 [6.4455426e-01]
 ...
 [9.1353059e-04]
 [4.8096180e-03]
 [5.6830722e-01]]


A rede esta confiante para algumas amostras (0.99 ou mais, ou 0.01 ou menos), mas menos confiante para outras (0.6, 0.4).

**Treinando o modelo do zero**

In [21]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [23]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [26]:
model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

Epoch 1/4
49/49 [==============================] - 0s 8ms/step - loss: 0.4573 - acc: 0.8190
Epoch 2/4
49/49 [==============================] - 0s 8ms/step - loss: 0.2594 - acc: 0.9103
Epoch 3/4
49/49 [==============================] - 0s 8ms/step - loss: 0.2005 - acc: 0.9285
Epoch 4/4
782/782 [==============================] - 1s 1ms/step - loss: 0.2935 - acc: 0.8846


In [28]:
print(results)

[0.2934575080871582, 0.8845599889755249]


Referênce: François Chollet. Deep Learning with Python. November 2017